References
- https://docs.llamaindex.ai/en/latest/examples/query_engine/knowledge_graph_query_engine.html#
- https://docs.llamaindex.ai/en/latest/examples/index_structs/knowledge_graph/KuzuGraphDemo.html#
- https://github.com/sroecker/LLM_AppDev-HandsOn
- [KG RAG query engine](https://colab.research.google.com/drive/1QpfgBvZZCg00WKNUgl6vZ1mgo-OfdynN#scrollTo=htIlL1nSiqFo)
- [Rebel + LlamaIndex Knowledge Graph Query Engine](https://colab.research.google.com/drive/1KPZ1GpqIwU4sEeYSRFEI_74rbuUZwX_B#scrollTo=5mHzFSTbPbWf)
- [Basic Knowledge Graph Index](https://gpt-index.readthedocs.io/en/latest/examples/index_structs/knowledge_graph/KnowledgeGraphDemo.html)
- [Knowledge Graph Index + Vector Index](https://gpt-index.readthedocs.io/en/latest/examples/index_structs/knowledge_graph/KnowledgeGraphIndex_vs_VectorStoreIndex_vs_CustomIndex_combined.html)
- [Knowledge Graph Index + Nebula](https://gpt-index.readthedocs.io/en/latest/examples/index_structs/knowledge_graph/NebulaGraphKGIndexDemo.html)
- [Querying existing Nebula KGs](https://gpt-index.readthedocs.io/en/latest/examples/query_engine/knowledge_graph_query_engine.html)
- [Knowledge Graph Index + Kuzu](https://gpt-index.readthedocs.io/en/latest/examples/index_structs/knowledge_graph/KuzuGraphDemo.html)

In [1]:
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)

## 1. Preparation
### 1.1 Prepare LLM

In [2]:
from llama_index.llms import Ollama

OLLAMA_HOST = 'localhost'
OLLAMA_MODEL = 'mistral'
llm = Ollama(model=OLLAMA_MODEL, base_url="http://"+OLLAMA_HOST+":11434")

In [3]:
from llama_index import ServiceContext
service_context = ServiceContext.from_defaults(
    llm=llm, 
    embed_model="local",
    # chunk_size=512
)

/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1.2 Prepare Graph Store

In [4]:
from llama_index.graph_stores import KuzuGraphStore

In [9]:
import shutil

shutil.rmtree("../data/kuzu-db-1", ignore_errors=True)

In [10]:
import kuzu

db = kuzu.Database("../data/kuzu-db-1")

In [11]:
graph_store = KuzuGraphStore(db)

In [12]:
from llama_index.storage.storage_context import StorageContext

storage_context = StorageContext.from_defaults(graph_store=graph_store)

## 2. Build the Knowledge Graph
### 2.1 Preprocess Data

In [13]:
from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()

documents = loader.load_data(pages=['Guardians of the Galaxy Vol. 3'], auto_suggest=False)

### 2.2 Extract Triplets and Save to Graph

In [14]:
from llama_index import KnowledgeGraphIndex

kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=10,
    include_embeddings=True,
    show_progress=True,
    # kg_triplet_extract_fn=extract_triplets
)

Processing nodes:   0%|                                                                                                                                               | 0/16 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


Processing nodes:   0%|                                                                                                                                               | 0/16 [00:29<?, ?it/s]


RuntimeError: Parameters must be a dict

## 3. Create VectorStoreIndex for RAG


In [10]:
from llama_index import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(
    documents,
    service_context=service_context
)

## 4. (Optional) Persist and Load from disk Llama Indexes
### 4.1. Persist

In [12]:
index.storage_context.persist(persist_dir='../data/storage_graph')

In [13]:
vector_index.storage_context.persist(persist_dir='../data/storage_vector')

### 4.2. Restore

In [14]:
from llama_index import load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir='../data/storage_graph', graph_store=graph_store)
kg_index = load_index_from_storage(
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=10,
    include_embeddings=True,
)

storage_context_vector = StorageContext.from_defaults(persist_dir='../data/storage_vector')
vector_index = load_index_from_storage(
    service_context=service_context,
    storage_context=storage_context_vector
)

INFO:llama_index.indices.loading:Loading all indices.
INFO:llama_index.indices.loading:Loading all indices.


## 5. Prepare for different query approaches

### 5.1 Graph RAG query engine

In [ ]:
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import KnowledgeGraphRAGRetriever

graph_rag_retriever = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
    verbose=True,
)

kg_rag_query_engine = RetrieverQueryEngine.from_args(
    graph_rag_retriever, service_context=service_context
)

In [16]:
# kg_rag_query_engine = kg_index.as_query_engine(
#     include_text=False,
#     retriever_mode="keyword",
#     response_mode="tree_summarize",
# )

### 5.2. Vector RAG query engine

In [17]:
vector_rag_query_engine = vector_index.as_query_engine()

### 5.3 Graph+Vector RAG query engine

In [28]:
# import QueryBundle
from llama_index import QueryBundle

# import NodeWithScore
from llama_index.schema import NodeWithScore

# Retrievers
from llama_index.retrievers import BaseRetriever, VectorIndexRetriever, KGTableRetriever

from typing import List


class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both Vector search and Knowledge Graph search"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        kg_retriever: KGTableRetriever,
        mode: str = "OR",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._kg_retriever = kg_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        kg_nodes = self._kg_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        kg_ids = {n.node.node_id for n in kg_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in kg_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(kg_ids)
        else:
            retrieve_ids = vector_ids.union(kg_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

In [29]:
from llama_index import get_response_synthesizer
from llama_index.query_engine import RetrieverQueryEngine

# create custom retriever
vector_retriever = VectorIndexRetriever(index=vector_index)
kg_retriever = KGTableRetriever(
    index=kg_index, retriever_mode="keyword", include_text=False
)
custom_retriever = CustomRetriever(vector_retriever, kg_retriever)

# create response synthesizer
response_synthesizer = get_response_synthesizer(
    service_context=service_context,
    response_mode="tree_summarize",
)

In [30]:
graph_vector_rag_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer,
)

## 6. Query with the engines

In [19]:
from IPython.display import Markdown, display

In [ ]:
response = query_engine.query(
    "Tell me about Peter Quill?",
)
display(Markdown(f"<b>{response}</b>"))

In [ ]:
response = await query_engine.aquery(
    "Tell me about Peter Quill?",
)
display(Markdown(f"<b>{response}</b>"))

In [20]:
response_graph_rag = kg_rag_query_engine.query("Tell me about Peter Quill.")

display(Markdown(f"<b>{response_graph_rag}</b>"))

INFO:llama_index.indices.knowledge_graph.retrievers:> No relationships found, returning nodes found by keywords.
INFO:llama_index.indices.knowledge_graph.retrievers:> No nodes found by keywords, returning empty response.


<b> Peter Quill, also known as Star-Lord, is a fictional character appearing in American comic books published by Marvel Comics. He was created by writer Dan Abnett and artist Andy Lanning, and first appeared in "Annihilators" #1 in July 2006.

Peter Quill is a human from Earth, born in 1982. He was abducted from Earth when he was a young boy and raised among the Ravagers, an intergalactic pirate gang. Quill became a skilled mercenary and thief, using the alias Star-Lord. He gained prominence when he led a team of heroes known as the Guardians of the Galaxy to save the universe from various threats.

Quill is known for his advanced weaponry, including the elementally powered Star-Lord gun and the Orb, an ancient relic that contains the power of a Celestial. He also has a sentient spaceship named the Milano, which can fly through space and time.

Quill's personality is described as being charming, rebellious, and sarcastic, with a strong sense of loyalty to those he cares about. He has been romantically involved with several characters in the Marvel Universe, including Gamora and Rocket Raccoon.

Quill made his cinematic debut in the 2014 film "Guardians of the Galaxy," portrayed by Chris Pratt. The character has since appeared in two sequels and other Marvel Cinematic Universe films.</b>

In [21]:
response_vector_rag = vector_rag_query_engine.query("Tell me about Peter Quill.")

display(Markdown(f"<b>{response_vector_rag}</b>"))

<b> Peter Quill, also known as Star-Lord, is a fictional character in the Marvel Cinematic Universe. He is portrayed by Chris Pratt in the films "Guardians of the Galaxy" (2014), "Guardians of the Galaxy Vol. 2" (2017), and "Avengers: Infinity War" and "Avengers: Endgame" (both 2018). Peter Quill is a human who was abducted from Earth as a child and grew up among extraterrestrial beings, becoming a space adventurer and eventually joining the titular Guardians of the Galaxy. He is known for his skill in combat and piloting spaceships, as well as his quirky sense of humor and love for classic Earth music. In the movies, he is depicted as a more traditional superhero compared to the Guardians, although not necessarily a hero. James Gunn, who directed the first two "Guardians" films, was fired by Disney in July 2018 over old tweets containing controversial remarks about rape and pedophilia, but there were efforts from Marvel and Disney to find a way for him to return to the franchise in some capacity. However, these negotiations ultimately failed and Gunn went on to work on other projects.</b>

In [31]:
response_graph_vector_rag = graph_vector_rag_query_engine.query("Tell me about Peter Quill.")

display(Markdown(f"<b>{response_graph_vector_rag}</b>"))

INFO:llama_index.indices.knowledge_graph.retrievers:> No relationships found, returning nodes found by keywords.
INFO:llama_index.indices.knowledge_graph.retrievers:> No nodes found by keywords, returning empty response.


<b> Peter Quill, also known as Star-Lord, is the leader of the Guardians of the Galaxy. He was abducted from Earth as a child and raised by a group of alien thieves and smugglers called the Ravagers. In the film "Guardians of the Galaxy Vol. 3," Quill is in a state of depression following the appearance of an alternate version of his deceased lover Gamora, who does not share the same affection for him as her older version did, affecting his leadership of the Guardians. Chris Pratt portrays Peter Quill in the film.</b>

## 7. Comparison and Conclusion

In [32]:
analysis = llm.complete(f"""
Compare the QA results on "Tell me about Peter Quill.", list the knowledge facts between them, to help evalute them. Output in markdown table.

Result from Graph: {response_graph_rag}
---
Result from Vector: {response_vector_rag}
---
Result Graph+Vector: {response_graph_vector_rag}
---

""")

In [33]:
display(Markdown(analysis.text))

 | Fact | Graph Result | Vector Result | Both Results |
| --- | --- | --- | --- |
| Character Name | Peter Quill / Star-Lord | Peter Quill / Star-Lord | Peter Quill / Star-Lord |
| Universe | Marvel Comics | Marvel Cinematic Universe | Marvel Universe |
| Creation | Created by Dan Abnett and Andy Lanning, first appeared in "Annihilators" #1 in July 2006 | Portrayed by Chris Pratt in films since 2014 | Created, first appearance in respective universes |
| Species | Human from Earth | Human who was abducted from Earth as a child and grew up among extraterrestrial beings | Human, abducted from Earth as a child, raised among aliens |
| Birth Year | Born in 1982 | N/A | Born in 1982 |
| Backstory | Raised among the Ravagers, skilled mercenary and thief, leads Guardians of the Galaxy | Skilled in combat and piloting spaceships, quirky sense of humor, love for classic Earth music, depicted as more traditional superhero compared to the Guardians | Raised among the Ravagers, became a space adventurer, joined the Guardians of the Galaxy |
| Weaponry | Star-Lord gun, Orb, Milano spaceship | N/A | Advanced weaponry, including Star-Lord gun and Orb, Milano spaceship |
| Personality | Charming, rebellious, sarcastic, strong sense of loyalty to those he cares about, romantically involved with Gamora and Rocket Raccoon | Quirky sense of humor, love for classic Earth music, depicted as a more traditional superhero compared to the Guardians | Charming, rebellious, sarcastic, romantic relationships with Gamora and Rocket Raccoon |
| Films | N/A | Appeared in "Guardians of the Galaxy" (2014), "Guardians of the Galaxy Vol. 2" (2017), "Avengers: Infinity War" (2018), and "Avengers: Endgame" (2018) | N/A, appears in films |
| Director | N/A | James Gunn was fired from the franchise over old tweets, but negotiations to return ultimately failed | N/A, no mention of director |
| Depiction in "Vol. 3" | In a state of depression following Gamora's appearance | N/A | In a state of depression following Gamora's appearance in "Guardians of the Galaxy Vol. 3" |